# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [1]:
#  !pip install syft

In [2]:
# try this project here!
import torch
import syft
from torch import optim, nn

W0722 17:08:27.759657 140149755836288 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0722 17:08:27.779470 140149755836288 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = syft.TorchHook(torch)

In [0]:
# A Toy Dataset
data = torch.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = torch.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# create worketrs
bob = syft.VirtualWorker(hook,"bob")
alice = syft.VirtualWorker(hook,"alice")
secure_worker = syft.VirtualWorker(hook,"secure_worker")

In [0]:
#  inform each workers to others optional for virtual worker need for realworld application
bob.add_workers([alice,secure_worker])
alice.add_workers([bob,secure_worker])
secure_worker.add_workers([bob,alice])

W0722 15:45:57.095317 140075321141120 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0722 15:45:57.098002 140075321141120 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0722 15:45:57.099905 140075321141120 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0722 15:45:57.101623 140075321141120 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0722 15:45:57.103509 140075321141120 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0722 15:45:57.105055 140075321141120 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:1>

In [0]:
bob

<VirtualWorker id:bob #objects:3>

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
for _ in range(10):
# model copy for each worker
  bob_model = model.copy().send(bob)
  alices_model = model.copy().send(alice)

# optimizer copy for each worker
  bob_opt = optim.SGD(params=bob_model.parameters(), lr=0.1)
  alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)
  
  for _ in range(10):
#     train bob
    bob_opt.zero_grad()
    bob_pred = bob_model(data_bob)
    bob_loss = ((bob_pred-target_bob)**2).sum()
    bob_loss.backward()
    bob_opt.step()
    bob_loss = bob_loss.get().data

#     train alices
    alices_opt.zero_grad()
    alices_pred = alices_model(data_alice)
    alices_loss = ((alices_pred - target_alice)**2).sum()
    alices_loss.backward()
    alices_opt.step()
    alices_loss = alices_loss.get().data

#   move both models to secure worker
  alices_model.move(secure_worker)
  bob_model.move(secure_worker)

#   set central model
  with torch.no_grad():
    model.weight.set_(((bob_model.weight.data + alices_model.bias.data) / 2).get())
    model.bias.set_(((bob_model.bias.data + alices_model.bias.data) / 2).get())

  print("bob:" + str(bob_loss) + "alice:" + str(alices_loss))

bob:tensor(0.0367)alice:tensor(0.0045)
bob:tensor(0.0053)alice:tensor(0.0008)
bob:tensor(0.0029)alice:tensor(0.0013)
bob:tensor(0.0025)alice:tensor(0.0011)
bob:tensor(0.0024)alice:tensor(0.0010)
bob:tensor(0.0024)alice:tensor(0.0009)
bob:tensor(0.0024)alice:tensor(0.0009)
bob:tensor(0.0024)alice:tensor(0.0009)
bob:tensor(0.0024)alice:tensor(0.0009)
bob:tensor(0.0024)alice:tensor(0.0009)


In [0]:
secure_worker._objects

{4717854057: Parameter containing:
 tensor([[-0.0851, -0.5983]], requires_grad=True),
 6819396721: Parameter containing:
 tensor([[-0.0370,  0.2523]], requires_grad=True),
 7128726850: Parameter containing:
 tensor([0.0238], requires_grad=True),
 7152004045: Parameter containing:
 tensor([0.4891], requires_grad=True),
 7318680239: Parameter containing:
 tensor([[0.0633, 0.4553]], requires_grad=True),
 13980357000: Parameter containing:
 tensor([0.7025], requires_grad=True),
 14287235123: Parameter containing:
 tensor([[-0.0394,  0.2407]], requires_grad=True),
 14872884466: Parameter containing:
 tensor([[-0.0371,  0.2520]], requires_grad=True),
 15069808316: Parameter containing:
 tensor([0.0245], requires_grad=True),
 21655257877: Parameter containing:
 tensor([[-0.0382,  0.2467]], requires_grad=True),
 21918435721: Parameter containing:
 tensor([0.4958], requires_grad=True),
 22370825765: Parameter containing:
 tensor([0.4832], requires_grad=True),
 24614787353: Parameter containing:

In [0]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [0]:
secure_worker._objects

{}

# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [0]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [0]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [0]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
# try this project here!
import random

In [0]:
Q = 98345769678960923

In [0]:
def encrypt(x, n_shares):
  shares = list()
  for i in range(n_shares - 1):
    shares.append(random.randint(0, Q))
  
  shares.append( Q - (sum(shares) % Q) + x)
  
  return tuple(shares)  

In [0]:
encrypt(5,3)

(72087670896753179, 30161407696685449, 94442460764483223)

In [0]:
def decrypt(x):
  return sum(x) % Q

In [0]:
decrypt(encrypt(5,3))

5

In [0]:
def add(x,y):
  z = list()
  for i in range(len(x)):
    z.append((x[i] + y[i]) % Q)
  return tuple(z)
  
  

In [0]:
x = encrypt((5),3)
y = encrypt(10,3)
z = add(x,y)
decrypt(z)

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(3.5)

35000

In [0]:
decode(35000)

3.5

In [0]:
x = encrypt(encode(5.5),2)
y = encrypt(encode(2.3),2)
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = torch.tensor([1,2,3,4,5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [0]:
bob._objects

{88711289105: tensor([2480406296973195402, 3953041399640757251, 1770296810131592713,
          467038501243238193, 3777796636830662538])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [0]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:702177566 -> bob:98642590081]
	-> (Wrapper)>[PointerTensor | me:33791338282 -> alice:91673352226]
	-> (Wrapper)>[PointerTensor | me:3995296874 -> secure_worker:88057873186]
	*crypto provider: me*

In [0]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [0]:
x

In [0]:
x = x.fix_prec()

In [0]:
x.child.child

In [0]:
y = x + x

In [0]:
y = y.float_prec()
y

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = torch.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [0]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [0]:
# A Toy Dataset
data = torch.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = torch.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# create worketrs
bob = syft.VirtualWorker(hook,"bob")
alice = syft.VirtualWorker(hook,"alice")
secure_worker = syft.VirtualWorker(hook,"secure_worker")

In [6]:
bob

<VirtualWorker id:bob #objects:0>

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:


# We encode everything
data = data.fix_precision().share(bob, alice, crypto_provider=secure_worker, requires_grad=True)
target = target.fix_precision().share(bob, alice, crypto_provider=secure_worker, requires_grad=True)
model = model.fix_precision().share(bob, alice, crypto_provider=secure_worker, requires_grad=True)



In [9]:
for _ in range(10):
# optimizer copy for each worker
  opt = optim.SGD(params=model.parameters(),lr=0.1).fix_precision()
  
  for _ in range(10):
#     Clean grad
    opt.zero_grad()
#   prediction
    logits = model(data)
#   calculate loss
    loss = ((logits - target)**2).sum()
#   backpropogation
    loss.backward()
#   optimize
    opt.step()

  print(loss.get().float_precision())

tensor([0.0460])
tensor([0.0030])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])
tensor([0.])


In [0]:
secure_worker._objects

{4717854057: Parameter containing:
 tensor([[-0.0851, -0.5983]], requires_grad=True),
 6819396721: Parameter containing:
 tensor([[-0.0370,  0.2523]], requires_grad=True),
 7128726850: Parameter containing:
 tensor([0.0238], requires_grad=True),
 7152004045: Parameter containing:
 tensor([0.4891], requires_grad=True),
 7318680239: Parameter containing:
 tensor([[0.0633, 0.4553]], requires_grad=True),
 13980357000: Parameter containing:
 tensor([0.7025], requires_grad=True),
 14287235123: Parameter containing:
 tensor([[-0.0394,  0.2407]], requires_grad=True),
 14872884466: Parameter containing:
 tensor([[-0.0371,  0.2520]], requires_grad=True),
 15069808316: Parameter containing:
 tensor([0.0245], requires_grad=True),
 21655257877: Parameter containing:
 tensor([[-0.0382,  0.2467]], requires_grad=True),
 21918435721: Parameter containing:
 tensor([0.4958], requires_grad=True),
 22370825765: Parameter containing:
 tensor([0.4832], requires_grad=True),
 24614787353: Parameter containing:

In [0]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

In [0]:
secure_worker._objects

{}